In [1]:
import csv
import json
import pathlib
import psycopg

In [2]:
SECRETS = json.load(open('../secrets.json', 'r'))
archives = pathlib.Path('../data-archive/us/pa/state/PA/')

In [3]:
directories = [d for d in archives.iterdir()]

In [62]:
def load_rows(path):
    with open(path, newline='\n') as csvfile:
        records = csv.reader(csvfile, delimiter=',')
        rows = [row for row in records]
        print(f'{path}: {len(rows)} loaded')
        return rows

def write_rows(table, rows):
    header = rows[0]
    print(", ".join(header))
    with cursor.copy(f'COPY staging.{table} FROM STDIN') as copy:
        row_count = 0
        for row in rows[1:]:
            copy.write_row(row)
            row_count += 1
        print(f'{table}: {row_count} processed')

In [55]:
rows = load_rows(pathlib.Path('../data-archive/us/pa/state/PA/2017-2018/PA_2017-2018_bill_actions.csv'))

../data-archive/us/pa/state/PA/2017-2018/PA_2017-2018_bill_actions.csv: 18947 loaded


In [56]:
type(rows)

list

In [57]:
print(rows[0])
print(rows[-1])

['id', 'bill_id', 'organization_id', 'description', 'date', 'classification', 'order']
['423a6569-60d0-4566-a9ff-3f126c57d475', 'ocd-bill/7cd4d381-b16c-4f37-89a0-978e7a4e0439', 'ocd-organization/0f78429d-ec86-4333-9338-17708fe11914', 'Approved by the Governor', '2017-10-30T04:00:00+00:00', "['executive-signature']", '32']


In [52]:
test_rows = [rows[0], rows[-1]]
test_rows

[['id',
  'bill_id',
  'organization_id',
  'description',
  'date',
  'classification',
  'order'],
 ['423a6569-60d0-4566-a9ff-3f126c57d475',
  'ocd-bill/7cd4d381-b16c-4f37-89a0-978e7a4e0439',
  'ocd-organization/0f78429d-ec86-4333-9338-17708fe11914',
  'Approved by the Governor',
  '2017-10-30T04:00:00+00:00',
  "['executive-signature']",
  '32']]

In [60]:
with psycopg.connect(f"host={SECRETS['db_host']} dbname={SECRETS['db_name']} user={SECRETS['db_user_name']} password={SECRETS['db_user_pwd']}") as conn:
    table = 'pa_monthly_bill_actions'
    header = rows[0]
    with conn.cursor() as cursor:
        with cursor.copy(f'COPY staging.{table} FROM STDIN') as copy:
            for row in rows[1:]:
                copy.write_row(row)

In [61]:
with psycopg.connect(f"host={SECRETS['db_host']} dbname={SECRETS['db_name']} user={SECRETS['db_user_name']} password={SECRETS['db_user_pwd']}") as conn:
    with conn.cursor() as cursor:
        cursor.execute(f'select * from staging.pa_monthly_bill_actions')
        print(cursor.fetchall())

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [44]:
with psycopg.connect(f"host={SECRETS['db_host']} dbname={SECRETS['db_name']} user={SECRETS['db_user_name']} password={SECRETS['db_user_pwd']}") as conn:
    with conn.cursor() as cursor:
        rows = test_rows
        header = test_rows[0]
        
        row_count = 0
        #print(", ".join(header))
        try:
            
            with cursor.copy(f'insert into staging.pa_monthly_bill_actions (id, bill_id, organization_id, description, date, classification, order) values STDIN') as copy:
                for row in rows[1:]:
                    copy.write_row(row)
                    row_count += 1
        except Exception as error:
            print(f'Error! {error}')

Error! syntax error at or near "order"
LINE 1: ...ganization_id, description, date, classification, order) val...
                                                             ^


In [63]:
def import_directory(directory):
    print(cursor)
    for f in directory.iterdir():
        if f.name.endswith('bill_actions.csv'):
            table = 'pa_monthly_bill_actions'
            rows = load_rows(f)
            write_rows(table, rows)
            print(f'{f} -> {table} complete')
        elif f.name.endswith('bill_sources.csv'):
            table = 'pa_monthly_bill_sources'
            rows = load_rows(f)
            write_rows(table, rows)
            print(f'{f} -> {table} complete')
        elif f.name.endswith('bill_sponsorships.csv'):
            table = 'pa_monthly_bill_sponsorships'
            rows = load_rows(f)
            write_rows(table, rows)
            print(f'{f} -> {table} complete')
        elif f.name.endswith('bill_version_links.csv'):
            table = 'pa_monthly_bill_version_links'
            rows = load_rows(f)
            write_rows(table, rows)
            print(f'{f} -> {table} complete')
        elif f.name.endswith('bill_versions.csv'):
            table = 'pa_monthly_bill_versions'
            rows = load_rows(f)
            write_rows(table, rows)
            print(f'{f} -> {table} complete')
        elif f.name.endswith('bills.csv'):
            table = 'pa_monthly_bills'
            rows = load_rows(f)
            write_rows(table, rows)
            print(f'{f} -> {table} complete')
        elif f.name.endswith('vote_counts.csv'):
            table = 'pa_monthly_vote_counts'
            rows = load_rows(f)
            write_rows(table, rows)
            print(f'{f} -> {table} complete')
        elif f.name.endswith('vote_people.csv'):
            table = 'pa_monthly_vote_people'
            rows = load_rows(f)
            write_rows(table, rows)
            print(f'{f} -> {table} complete')
        elif f.name.endswith('vote_sources.csv'):
            table = 'pa_monthly_vote_sources'
            rows = load_rows(f)
            write_rows(table, rows)
            print(f'{f} -> {table} complete')
        elif f.name.endswith('votes.csv'):
            table = 'pa_monthly_votes'
            rows = load_rows(f)
            write_rows(table, rows)
            print(f'{f} -> {table} complete')

In [65]:
with psycopg.connect(f"host={SECRETS['db_host']} dbname={SECRETS['db_name']} user={SECRETS['db_user_name']} password={SECRETS['db_user_pwd']}") as conn:
    with conn.cursor() as cursor:
        for directory in directories:
            print(f'Processing {directory}')
            import_directory(directory)

Processing ../data-archive/us/pa/state/PA/2017-2018
<psycopg.Cursor [no result] [IDLE] (host=172.16.0.56 user=eron database=datalake) at 0x75fbf343e400>
../data-archive/us/pa/state/PA/2017-2018/PA_2017-2018_vote_counts.csv: 11944 loaded
id, vote_event_id, option, value
pa_monthly_vote_counts: 11943 processed
../data-archive/us/pa/state/PA/2017-2018/PA_2017-2018_vote_counts.csv -> pa_monthly_vote_counts complete
../data-archive/us/pa/state/PA/2017-2018/PA_2017-2018_bill_version_links.csv: 21450 loaded
id, media_type, url, version_id
pa_monthly_bill_version_links: 21449 processed
../data-archive/us/pa/state/PA/2017-2018/PA_2017-2018_bill_version_links.csv -> pa_monthly_bill_version_links complete
../data-archive/us/pa/state/PA/2017-2018/PA_2017-2018_bill_versions.csv: 6467 loaded
id, bill_id, note, date, classification, extras
pa_monthly_bill_versions: 6466 processed
../data-archive/us/pa/state/PA/2017-2018/PA_2017-2018_bill_versions.csv -> pa_monthly_bill_versions complete
../data-archi